# Hierarchical Reasoning Model

Revisión informal de "_Hierarchical Reasoning Model_" (_HRM_).

Arquitectura publicada en junio de 2025.

- [https://arxiv.org/pdf/2506.21734](https://arxiv.org/pdf/2506.21734)

Permite la creación de modelos de reducido tamaño que resuelven tareas específicas con pocos ejemplos y sin preentrenamiento.

## 1. Arquitectura

La arquitectura se compone de dos módulos.

```
High-Level (H)
```

Un módulo _H_ de alto nivel (_High-Level_) para el razonamiento abstracto que dirige la estrategia.

```
Low-Level (L)
```

Y un módulo _L_ de bajo nivel (_Low-Level_) para los cálculos detallados, como búsquedas intensivas o refinamientos.

El _paper_ presenta resultados experimentales de modelos siguiendo esta arquitectura con apenas 27 millones de parámetros y entrenados con unos escasos 1.000 ejemplos que son capaces de resolver _sudokus_ o encontrar la ruta óptima de salida de laberintos.

## 2. Modelo

### 2.1. Componentes

El modelo propuesto por la arquitectura se estructura en cuatro componentes entrenables.

- Una red de entrada $f_I(\cdot ; \theta_I)$.

- Un módulo recurrente de bajo nivel $f_L(\cdot ; \theta_L)$.

- Un módulo recurrente de alto nivel $f_H(\cdot ; \theta_H)$.

- Una red de salida $f_O(\cdot ; \theta_O)$.

Donde $\theta$ representa los parámetros entrenables de cada componente.

### 2.2. Ciclos

```
N x High-Level x T x Low-Level
```

El modelo opera realizando $N$ ciclos de alto nivel, cada uno de ellos compuesto de $T$ pasos de bajo nivel.

- $ i = 1, \ldots, N \times T $

Los módulos _L_ y _H_ tienen un estado oculto $z_L^i$ y $z_H^i$, que es inicializado con un vector $z_L^0$ y $z_H^0$, respectivamente.

### 2.3. Forward Pass

La red de entrada proyecta un vector de entrada $x$ en otro vector $\tilde{x}$.

- $ \tilde{x} = f_I(x ; \theta_I) $

En cada paso $i$ el módulo _L_ se actualiza en función de su estado anterior, el estado anterior de _H_, y la entrada.

- $ z_L^i = f_L(z_L^{i-1}, z_H^{i-1}, \tilde{x}; \theta_L) $

Y el módulo _H_ se actualiza cada $T$ pasos en función de su estado anterior, y el estado de _L_.

- $ z_H^i = \begin{cases}
f_H(z_H^{i-1}, z_L^i; \theta_H) & \text{si} & (i \mod T) = 0,
\\ z_H^{i-1} & \text{en} & \text{caso contrario.}
\end{cases} $

Después de $N$ ciclos completos, de $T$ pasos cada uno, la red de salida obtiene un vector con la predición $\tilde{y}$ en función del estado del módulo _H_.

- $ \tilde{y} = f_O(z_H^{NT} ; \theta_O) $

## 3. Training

### 3.1. Backward Pass

Una red recurrente tradicional requiere almacenar los estados ocultos de cada paso para poder combinarlos posteriormente con los gradientes durante el _backward pass_, lo que impone $O(T)$ requerimientos de memoria para $T$ pasos.

_HRM_ utiliza únicamente el estado oculto de cada módulo en el último paso, lo que impone sólo $O(1)$ requerimientos de memoria.

El marco teórico del _paper_ establece que, si el módulo _L_ converge a un punto fijo local $z_L^\star$, entonces, aplicando resultados conocidos de _Deep Equilibrium Models_ (_DEQ_) basados en _Implicit Function Theorem_ (_IFT_), se pueden calcular aproximaciones a los gradientes utilizando sólo los últimos estados.

- $ \dfrac{\partial{z_L^\star}}{\partial{\theta_L}} \approx \dfrac{\partial{f_L}}{\partial{\theta_L}}, \quad
\dfrac{\partial{z_L^\star}}{\partial{\theta_I}} \approx \dfrac{\partial{f_L}}{\partial{\theta_I}} $

Y de igual forma para el módulo _H_.

- $ \dfrac{\partial{z_H^\star}}{\partial{\theta_H}} \approx \dfrac{\partial{f_H}}{\partial{\theta_H}}, \quad
\dfrac{\partial{z_H^\star}}{\partial{\theta_L}} \approx \dfrac{\partial{f_H}}{\partial{z_L^\star}} \cdot \dfrac{\partial{z_L^\star}}{\partial{\theta_L}}, \quad
\dfrac{\partial{z_H^\star}}{\partial{\theta_I}} \approx \dfrac{\partial{f_H}}{\partial{z_L^\star}} \cdot \dfrac{\partial{z_L^\star}}{\partial{\theta_I}} $

### 3.2. Deep Supervision

_HRM_ utiliza un mecanismo de _deep supervision_ para guiar la actualización de los pesos.

En vez de aplicar un único _forward pass_ para cada muestra $(x, y)$ del conjunto de entrenamiento, se ejecutan $M$ _forward  pass_ llamados _segmentos_.

- $ m \in \{1, \ldots, M\} $

De forma que al final de cada segmento se obtiene un estado final para los módulos _H_ y _L_.

- $ z^m = (z_H^{mNT}, z_L^{mNT}) $

El mecanismo de _deep supervision_ calcula el estado final y la salida predicha mediante un _forward pass_ con el estado final del segmento anterior y la entrada actual.

- $ (z^m, \hat{y}^m) \leftarrow \operatorname{HRM}(z^{m-1}, x ; \theta) $

A continuación calcula la pérdida para el segmento actual.

- $ L^m \leftarrow \operatorname{LOSS}(\hat{y}^m, y) $

Y actualiza los pesos con los gradientes.

- $ \theta \leftarrow \operatorname{OptimizerStep}(\theta, \nabla_{\theta}L^m) $

De esta forma la influencia en el error cometido se aplica segmento a segmento, sin propagar el error a través de todos los segmentos, y actuando como un mecanismo de regularización que tiene en cuenta únicamente las activaciones más recientes.

### 3.3. Adaptive Computational Time

_HRM_ utiliza el algoritmo _Q-learning_ para determinar el número de segmentos a ejecutar.

El número máximo $M_\text{max}$ de segmentos se establece mediante un hiperparámetro del modelo. Y el número mínimo $M_\text{min}$ de segmentos se determina estocásticamente en función de un hiperparámetro $\epsilon$ del modelo. Para $\epsilon$, $M_\text{min}$ se establece tomando una muestra usando una distribución de probabilidad uniforme sobre el conjunto $\{2, \ldots, M_\text{max}\}$, y para $1 - \epsilon$, $M_\text{min}$ se establece a $1$.

_Q-learning_ funciona en base a una tabla de estados y acciones que se pueden realizar sobre dichos estados. Los valores sobre la tabla determinan la calidad por la ejecución de cada acción en cada estado.

_HRM_ define dos posibles acciones: "_halt_" (_parar_) y "_continue_" (_continuar_), y las recompensas para cada acción tras la ejecución de cada segmento $m$.

- $ \hat{G}^m = (\hat{G}_\text{halt}^m, \hat{G}_\text{continue}^m) $

La acción "_halt_" termina la ejecución de segmentos, y proporciona una recompensa condicionada de $0$ ó $1$ en función de si la salida predicha se corresponde con la esperada.

- $ \hat{G}_\text{halt}^m = \textbf{1}\{\hat{y}^m = y\} $

La acción "_continue_" continúa con la ejecución del siguiente segmento, y proporciona una recompensa de $0$.

- $ \hat{G}_\text{continue}^m = \begin{cases}
\hat{Q}_\text{halt}^{m+1} & \text{si} & m \ge M_{max},
\\ \operatorname{max}(\hat{Q}_\text{halt}^{m+1}, \hat{Q}_\text{continue}^{m+1}) & \text{en} & \text{caso contrario}.
\end{cases} $

Lo que quiere decir la expresión es que la acción "_halt_" se selecciona cuando se alcanza $M_\text{max}$ segmentos ejecutados, o $\hat{Q}_\text{halt}$ es mayor que $\hat{Q}_\text{continue}$, siempre y cuando se hayan ejecutado al menos $M_\text{min}$ segmentos.

Los valores $\hat{Q}^m$ correspondientes a las dos posibles acciones se calculan utilizando el estado final del módulo _H_.

- $ \hat{Q}^m = (\hat{Q}_\text{halt}^m, \hat{Q}_\text{continue}^m) = \sigma\left(\theta_Q^\intercal z_H^{mNT}\right) $

Donde $\theta_Q^\intercal$ es la traspuesta de la matriz de pesos entrenable de la capa _Q-Head_ que implementa _Q-learning_, y $\sigma$ la función sigmoide aplicada elemento a elemento.

### 3.4. Loss

_HRM_ calcula la pérdida final para cada segmento teniendo en cuenta la pérdida calculada por el mecanismo de _deep supervision_ y la pérdida de la capa _Q-Head_.

- $ L_\text{ACT}^m = \operatorname{LOSS}(\hat{y}^m, y) + \operatorname{BinaryCrossEntropy}(\hat{Q}^m, \hat{G}^m) $

## 4. Implementación

### 4.1. sequence-to-sequence

```
Input ─> I ─> L ─> H ─> O ─> Output
```

El modelo se implementa como una arquitectura _sequence-to-sequence_ que recibe una secuencia de _tokens_ $x$ de longitud $l$ y emite una secuencia de _tokens_ $y$ de longitud $l'$.

- $ x = (x_1, \ldots, x_l) $

- $ y = (y_1, \ldots, y_{l'}) $

### 4.2. Input Network

```
Input ─> Embedding ─> L ─> H ─> O ─> Output
```

La red de entrada $f_I$ se implementa con una capa de _embedding_.

### 4.3. Recurrent Modules

```
Input ─> Embedding ─> Transformer ─> Transformer ─> O ─> Output
```

Los dos módulos recurrentes se implementan como _encoders_, ambos siguiendo la misma arquitectura _Transformer_, con las mismas dimensiones.

Los estados internos $z$ son inicializados tomando muestras con una distribución normal de media $0$ y desviación típica $1$ dentro del rango $[-2, 2]$.

Y la implementación utiliza las técnicas más habituales en desarrollos actuales como _Rotary Positional Encoding_, _Gated Linear Units_, _RMSNorm_, y prescinde de los términos de _bias_ en las capas lineales.

### 4.4. Output Network

```
Input ─> Embedding ─> Transformer ─> Transformer ─> SoftMax ─> Output
```

La red de salida $f_O$ se implementa aplicando la función $\operatorname{softmax}$.

- $ f_O(z; \theta_O) = \operatorname{softmax}(\theta_O z) $

O alternativamente la función $\operatorname{stablemax}$ para experimentos con un número reducido de muestras de entrenamiento.

- $ \operatorname{stablemax}(x_i) = \dfrac{s(x_i)}{\sum\limits_j s(x_j)} $

Siendo:

- $ s(x) = \begin{cases}
x + 1 & \text{si} & x \ge 0,
\\ \dfrac{1}{1 - x} & \text{si} & x < 0.
\end{cases} $

La salida es la distribución de probabilidad $\hat{y}$ calculada mediante $softmax$ a partir de los estados internos $z$.

### 4.5. Loss

```
Input ─> Embedding ─> Transformer ─> Transformer ─> SoftMax ─> Loss ─> Output
```

La pérdida a la salida de la red se calcula promediando para todos los _tokens_ de la secuencia de salida.

- $ \operatorname{LOSS}(\hat{y}, y) = \dfrac{1}{l'} \sum\limits_{i=1}^{l'} \log p(y_i) $

Siendo $p(y_i)$ la probabilidad que la distribución de salida $\hat{y}_i$ asigna al _token_ $y_i$.